In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dfClimate = pd.read_csv('/content/drive/MyDrive/Data Science Class/Final Project/Data/FAOSTAT_data_en_11-19-2024 (1).csv')

This first part of the project involes extensive data cleaning.

In [ ]:
#remove rows with years before 1990 in the Year column
dfClimate = dfClimate[dfClimate['Year'] >= 1990]
dfClimate.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Months Code,Months,Year Code,Year,Unit,Value,Flag,Flag Description
29,ET,Temperature change on land,840,United States of America,7271,Temperature change,7001,January,1990,1990,°c,2.852,E,Estimated value
30,ET,Temperature change on land,840,United States of America,7271,Temperature change,7001,January,1991,1991,°c,0.230,E,Estimated value
31,ET,Temperature change on land,840,United States of America,7271,Temperature change,7001,January,1992,1992,°c,2.224,E,Estimated value
32,ET,Temperature change on land,840,United States of America,7271,Temperature change,7001,January,1993,1993,°c,0.634,E,Estimated value
33,ET,Temperature change on land,840,United States of America,7271,Temperature change,7001,January,1994,1994,°c,0.357,E,Estimated value


In [ ]:
#remove the first seven columns from dfClimate
dfClimate = dfClimate.iloc[:, 7:]
dfClimate.head()

,Months,Year Code,Year,Unit,Value,Flag,Flag Description
29,January,1990,1990,°c,2.852,E,Estimated value
30,January,1991,1991,°c,0.230,E,Estimated value
31,January,1992,1992,°c,2.224,E,Estimated value
32,January,1993,1993,°c,0.634,E,Estimated value
33,January,1994,1994,°c,0.357,E,Estimated value


In [ ]:
#remove the Flag column and the Flag Description column
dfClimate = dfClimate.drop(columns=['Flag', 'Flag Description'])
dfClimate.head()

,Months,Year Code,Year,Unit,Value
29,January,1990,1990,°c,2.852
30,January,1991,1991,°c,0.230
31,January,1992,1992,°c,2.224
32,January,1993,1993,°c,0.634
33,January,1994,1994,°c,0.357


In [ ]:
#remove the second column
dfClimate = dfClimate.iloc[:, 2:]
dfClimate.head()

,Year,Unit,Value
29,1990,°c,2.852
30,1991,°c,0.230
31,1992,°c,2.224
32,1993,°c,0.634
33,1994,°c,0.357


In [ ]:
#make the year column a data time column that has dates like xx-xx-xxxx with the current entery in the year column as the year
dfClimate['Year'] = pd.to_datetime(dfClimate['Year'], format='%Y')
dfClimate.head()

,Year,Unit,Value
29,1990-01-01,°c,2.852
30,1991-01-01,°c,0.230
31,1992-01-01,°c,2.224
32,1993-01-01,°c,0.634
33,1994-01-01,°c,0.357


In [ ]:
#make the Year column into day-month-year
dfClimate['Year'] = dfClimate['Year'].dt.strftime('%d-%m-%Y')
dfClimate.head()

,Year,Unit,Value
29,01-01-1990,°c,2.852
30,01-01-1991,°c,0.230
31,01-01-1992,°c,2.224
32,01-01-1993,°c,0.634
33,01-01-1994,°c,0.357


In [ ]:
dfGHG = pd.read_csv('/content/drive/MyDrive/Data Science Class/Final Project/Data/Carbon_(CO2)_Emissions_by_Country.csv')
dfGHG.tail()

,Country,Region,Date,Kilotons of Co2,Metric Tons Per Capita
5672,Zimbabwe,Africa,01-01-2004,9770.0,0.80
5673,Zimbabwe,Africa,01-01-2007,9760.0,0.78
5674,Zimbabwe,Africa,01-01-2010,9600.0,0.75
5675,Zimbabwe,Africa,01-01-2009,7750.0,0.61
5676,Zimbabwe,Africa,01-01-2008,7600.0,0.61


In [ ]:
#filer by only the United States in the Country column
dfGHG = dfGHG[dfGHG['Country'] == 'United States']
dfGHG.tail()

,Country,Region,Date,Kilotons of Co2,Metric Tons Per Capita
5432,United States,Americas,01-01-1992,4879630.00,19.02
5433,United States,Americas,01-01-1990,4844520.00,19.41
5434,United States,Americas,01-01-2017,4819370.12,14.82
5435,United States,Americas,01-01-2019,4817720.21,14.67
5436,United States,Americas,01-01-1991,4807500.00,19.00


In [ ]:
#sort by date data that looks like xx-xx-xxxx by date
dfGHG = dfGHG.sort_values(by='Date')
dfGHG.head()


,Country,Region,Date,Kilotons of Co2,Metric Tons Per Capita
5433,United States,Americas,01-01-1990,4844520.0,19.41
5436,United States,Americas,01-01-1991,4807500.0,19.00
5432,United States,Americas,01-01-1992,4879630.0,19.02
5427,United States,Americas,01-01-1993,4995210.0,19.22
5426,United States,Americas,01-01-1994,5066810.0,19.26


In [ ]:
#remove the first two columns of dfGHG
dfGHG = dfGHG.iloc[:, 2:]
dfGHG.head()

,Date,Kilotons of Co2,Metric Tons Per Capita
5433,01-01-1990,4844520.0,19.41
5436,01-01-1991,4807500.0,19.00
5432,01-01-1992,4879630.0,19.02
5427,01-01-1993,4995210.0,19.22
5426,01-01-1994,5066810.0,19.26


In [ ]:
#change the Date column header to Year
dfGHG = dfGHG.rename(columns={'Date': 'Year'})
dfGHG.head()

,Year,Kilotons of Co2,Metric Tons Per Capita
5433,01-01-1990,4844520.0,19.41
5436,01-01-1991,4807500.0,19.00
5432,01-01-1992,4879630.0,19.02
5427,01-01-1993,4995210.0,19.22
5426,01-01-1994,5066810.0,19.26


In [ ]:
#comine the two dataframes based on the year column
dfClimateGHG = pd.merge(dfClimate, dfGHG, on='Year')
dfClimateGHG.head()


,Year,Unit,Value,Kilotons of Co2,Metric Tons Per Capita
0,01-01-1990,°c,2.852,4844520.0,19.41
1,01-01-1991,°c,0.230,4807500.0,19.00
2,01-01-1992,°c,2.224,4879630.0,19.02
3,01-01-1993,°c,0.634,4995210.0,19.22
4,01-01-1994,°c,0.357,5066810.0,19.26


In [ ]:
#reindex with the year column then drop the year column
dfClimateGHG = dfClimateGHG.set_index('Year')
dfClimateGHG.head()

,Unit,Value,Kilotons of Co2,Metric Tons Per Capita
Year,,,,
01-01-1990,°c,2.852,4844520.0,19.41
01-01-1991,°c,0.230,4807500.0,19.00
01-01-1992,°c,2.224,4879630.0,19.02
01-01-1993,°c,0.634,4995210.0,19.22
01-01-1994,°c,0.357,5066810.0,19.26


In [ ]:
#take the difference between every row of the third column and create a new column and calculate the percent change
dfClimateGHG['Difference CO2'] = dfClimateGHG['Kilotons of Co2'].diff()
dfClimateGHG['Percent Change CO2'] = dfClimateGHG['Kilotons of Co2'].pct_change()
dfClimateGHG.head()


,Unit,Value,Kilotons of Co2,Metric Tons Per Capita,Difference CO2,Percent Change CO2
Year,,,,,,
01-01-1990,°c,2.852,4844520.0,19.41,NaN,NaN
01-01-1991,°c,0.230,4807500.0,19.00,-37020.0,-0.007642
01-01-1992,°c,2.224,4879630.0,19.02,72130.0,0.015004
01-01-1993,°c,0.634,4995210.0,19.22,115580.0,0.023686
01-01-1994,°c,0.357,5066810.0,19.26,71600.0,0.014334


In [ ]:
#take the difference between every row of the third column and create a new column and calculate the percent change
dfClimateGHG['Difference Temp'] = dfClimateGHG['Value'].diff()
dfClimateGHG['Percent Change Temp'] = dfClimateGHG['Value'].pct_change()
dfClimateGHG.head()

,Unit,Value,Kilotons of Co2,Metric Tons Per Capita,Difference CO2,Percent Change CO2,Difference Temp,Percent Change Temp
Year,,,,,,,,
01-01-1990,°c,2.852,4844520.0,19.41,NaN,NaN,NaN,NaN
01-01-1991,°c,0.230,4807500.0,19.00,-37020.0,-0.007642,-2.622,-0.919355
01-01-1992,°c,2.224,4879630.0,19.02,72130.0,0.015004,1.994,8.669565
01-01-1993,°c,0.634,4995210.0,19.22,115580.0,0.023686,-1.590,-0.714928
01-01-1994,°c,0.357,5066810.0,19.26,71600.0,0.014334,-0.277,-0.436909


In [ ]:
#drop nan
dfClimateGHG = dfClimateGHG.dropna()
dfClimateGHG.head()

,Unit,Value,Kilotons of Co2,Metric Tons Per Capita,Difference CO2,Percent Change CO2,Difference Temp,Percent Change Temp
Year,,,,,,,,
01-01-1991,°c,0.230,4807500.0,19.00,-37020.0,-0.007642,-2.622,-0.919355
01-01-1992,°c,2.224,4879630.0,19.02,72130.0,0.015004,1.994,8.669565
01-01-1993,°c,0.634,4995210.0,19.22,115580.0,0.023686,-1.590,-0.714928
01-01-1994,°c,0.357,5066810.0,19.26,71600.0,0.014334,-0.277,-0.436909
01-01-1995,°c,1.838,5117040.0,19.22,50230.0,0.009914,1.481,4.148459


In [ ]:
#define for use in graphs
tons = '\u0394 Tons'
celsius ='\u0394C\N{DEGREE SIGN}'

The graph below is change in degrees celsius and change in tons of carbon dioxide emissions plotted on the same graph. We observed that a change in carbon dioxide emissions is followed by a change in temperature after two to five years. This is reasonable because climate change is a slow process that happens over years not months or days. Carbon dioxide  emitted from someone’s tailpipe probably does not instantly affect the temperature which explains the lag in the data.


In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=dfClimateGHG.index, y=dfClimateGHG['Difference Temp'], name=celsius),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=dfClimateGHG.index, y=dfClimateGHG['Difference CO2'], name=tons),
    secondary_y=True,
)

# Update layout for the y-axes
fig.update_layout(
    title_text="Difference in Yearly Temperature vs Difference in Yearly CO2 Emissions",
    yaxis=dict(title="Difference in Temperature "+"("+celsius+")"),  # Left y-axis
    yaxis2=dict(title="Difference in Yearly CO2 Emissions "+"("+tons+")", overlaying="y", side="right"),  # Right y-axis
)
fig.show()


In [ ]:
def linear_regression(X, y):
  X = sm.add_constant(X)
  model = sm.OLS(y, X).fit()
  return(model)

In [ ]:
print(linear_regression(dfClimateGHG['Difference CO2'],dfClimateGHG['Difference Temp']).summary())

                            OLS Regression Results                            
Dep. Variable:        Difference Temp   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.6675
Date:                Sun, 08 Dec 2024   Prob (F-statistic):              0.421
Time:                        04:05:03   Log-Likelihood:                -47.668
No. Observations:                  29   AIC:                             99.34
Df Residuals:                      27   BIC:                             102.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.0344      0.241     -0.

This is a graph with difference in yearly carbon dioxide emissions as the independent variable and difference in yearly temperature as the dependent variable, with the line of best fit plotted over the data points. This graph explains our low p-value, as our data does not follow the line. Additionally, the slope of the line is negative, which implies that increasing carbon dioxide dioxide correlates with a decrease in temperature, which is illogical.


In [ ]:
#linear regression line
model=linear_regression(dfClimateGHG['Difference CO2'],dfClimateGHG['Difference Temp'])
slope = model.params[1]
intercept = model.params[0]
x_values = dfClimateGHG['Difference CO2']
y_values = dfClimateGHG['Difference Temp']

# display using plotly
fig = px.scatter(dfClimateGHG, x='Difference CO2', y='Difference Temp', title='Difference in Yearly Temperature vs Difference in Yearly CO2 Emissions')
fig.add_trace(go.Scatter(x=x_values, y=slope*x_values + intercept, mode='lines', name='Linear Regression Line'))
fig.update_layout(xaxis_title='Difference in Yearly CO2 Emissions '+"("+tons+")", yaxis_title='Difference in Yearly Temperature'+"("+celsius+")")
fig.show()



<ipython-input-157-86b1b5e53a6c>:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-157-86b1b5e53a6c>:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
def crosscorr(datax, datay, lag=0):
    return datax.corr(datay.shift(lag))

# Calculate cross-correlation for various lags
lags = range(-10, 11)  # Check lags from -10 to +10
rs = [crosscorr(dfClimateGHG['Difference Temp'], dfClimateGHG['Difference CO2'], lag) for lag in lags]

# Find the lag with the maximum correlation
delay = lags[np.argmax(rs)]
print(f"The delay is: {delay}")


The delay is: -5


In [ ]:
dfClimateGHG['Shifted_Temp'] = dfClimateGHG['Difference Temp'].shift(periods= delay, fill_value=0)
print(len(dfClimateGHG))
#define rows to keep
keep=len(dfClimateGHG)+delay
print(keep)
#keep the first keep rows
dfClimateGHG = dfClimateGHG.iloc[:keep]
len(dfClimateGHG)

29
24


24

In this graph we shifted the temperature graph by 5 years to showcase how the delay lines up and dropped 5 years off the end of both graphs. This graph shows how the CO2 emissions would affect the temperature if the effects were instantaneous. A bunch of the peaks and valleys line up pretty perfectly near the beginning and for some reason near the end there is less correlation which we are still unable to explain.


In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=dfClimateGHG.index, y=dfClimateGHG['Shifted_Temp'], name="Shifted "+celsius),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=dfClimateGHG.index, y=dfClimateGHG['Difference CO2'], name=tons+" Co2"),
    secondary_y=True,
)

# Update layout for the y-axes
fig.update_layout(
    title_text="Difference in Yearly Temperature(Shifted by 5 Years) vs Difference in Yearly CO2 Emissions",
    yaxis=dict(title="Difference in Temperature "+"("+celsius+")"),  # Left y-axis
    yaxis2=dict(title="Difference in Yearly CO2 Emissions "+"("+tons+")", overlaying="y", side="right"),  # Right y-axis
)

fig.show()

In [ ]:
#Model results
print(linear_regression(dfClimateGHG['Difference CO2'], dfClimateGHG['Shifted_Temp']).summary())

                            OLS Regression Results                            
Dep. Variable:           Shifted_Temp   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.623
Date:                Sun, 08 Dec 2024   Prob (F-statistic):              0.216
Time:                        04:05:04   Log-Likelihood:                -36.259
No. Observations:                  24   AIC:                             76.52
Df Residuals:                      22   BIC:                             78.87
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.0190      0.234     -0.


This is a graph with the difference in yearly carbon dioxide emissions as the independent variable and the shifted difference in yearly temperature as the dependent variable, with the line of best fit plotted over the data points. This graph explains our medium p-value, as our data does not follow the line well but there appears to be some sort of collocation which explains our lower p-value, although not enough to be statistically significant. Additionally, the slope of the line is positive, which implies that increasing carbon dioxide dioxide correlates with an increase in temperature, which shows that this is likely closer to the truth than the previous analysis.


In [ ]:
# display using plotly
model=linear_regression(dfClimateGHG['Difference CO2'], dfClimateGHG['Shifted_Temp'])
x_values = dfClimateGHG['Difference CO2']
slope = model.params[1]
intercept = model.params[0]

fig = px.scatter(dfClimateGHG, x='Difference CO2', y='Shifted_Temp', title='Difference in Yearly Temperature vs Difference in Yearly CO2 Emissions')
fig.add_trace(go.Scatter(x=x_values, y=slope*x_values + intercept, mode='lines', name='Linear Regression Line'))
fig.update_layout(xaxis_title='Difference in Yearly CO2 Emissions '+"("+tons+")", yaxis_title='Difference in Yearly Temperature (Shifted)'+"("+celsius+")")
fig.show()

<ipython-input-162-115b44718a45>:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-162-115b44718a45>:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

